# Prioritizarion Playground

experimentation prior to finalization of weight of each factor

In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)

## Getting Data

In [12]:
clients = pd.read_csv('fake_CRM_data/clients.csv')
policies = pd.read_csv('fake_CRM_data/policies.csv')

In [13]:
clients.head(3)

,client_name,client_id,account_type,address,phone_number,email,branch_id,servicing_broker_id,status,issued_on,updated_on
0,Julie Elliott,C001611,Commercial,"7062 Jessica Springs, Malonemouth, NY 79009",471-376-6118x33574,julie.elliott@black-walter.com,B_US00590,S006413,Prospect,2006-05-22,2025-11-28
1,Kyle Perez,C002570,Personal,"929 Hoffman Points Suite 056, Michaelview, AZ ...",+1-822-213-8290,kyle.perez@myers-harris.org,B_US00583,S006148,Prospect,2006-05-19,2025-11-28
2,Allison Allen,C003734,Personal,"2985 Mike Island Apt. 600, South Benjamin, AZ ...",001-237-680-4133x1814,allison.allen@franklin.org,B_US00161,S009090,Prospect,2006-04-25,2025-11-28


In [14]:
policies.head(3)

,policy_id,client_id,line_of_business,issuing_carrier,issued_on,expiry_date,premium_amount,premium_bill_to,renewal_status,last_updated,broker_commision_pct
0,P003944,C001611,Contracting civil engineer,Murray PLC Insurance Inc.,2006-05-22,2026-05-22,4398.737534,Premium Finance Company,New,2025-11-28,15.536213
1,P001615,C002570,Scientific laboratory technician,"Nelson, Ward and Mcdowell Insurance Inc.",2006-05-19,2026-05-19,693.891941,Insured,Marketing,2025-11-28,18.766720
2,P004749,C003734,Hydrologist,Daniels-Brady Insurance Inc.,2006-04-25,2026-04-25,220.460744,Mortgagee,Renewal Quoted,2025-11-28,18.419806


## Preprocessing

### to Datetime

In [ ]:
###

## Getting Scores

In [21]:
today = clients.updated_on[0]
today

'2025-11-28'

In [22]:
pd.to_datetime(clients.updated_on)

0     2025-11-28
1     2025-11-28
2     2025-11-28
3     2025-11-28
4     2025-11-28
         ...    
995   2025-11-28
996   2025-11-28
997   2025-11-28
998   2025-11-28
999   2025-11-28
Name: updated_on, Length: 1000, dtype: datetime64[ns]